<div id="instructions" style="border-radius: 5px; background-color:#f5f5f5;" >
<h1>Instructions</h1>
<p>Look for the <b>4 Your Turn</b> sections to complete the code and/or answer questions.<p>
</div>

# Lesson 7 - Theory of Regression and Regularization

In this notebook we will explore the mathematical basis of linear statistical models. The emphasis is on the ubiquitious problem of **model overfitting** or **model over-parameterizaton**. 

Overfitting (or over-parameterization) of machine learning models arrises in any case where the number of model parameters exceeds the effective dimensions of the feature set. This is most often the result of linear dependency between the features. However, using too complex a model can lead to similar problems. In the extreme case, imagine a model with as many free parameters as training cases. This model might fit the training data perfectly, but will show unstable and unexpected results when used for other data. In machine learning terminology, we say that such an unstable model does not **generalize**. 

Many methods have been developed and continue to be developed to deal with over-paramterized or **ill-posed** machine learning models. In particular, in this notebook we will explore three methods for stabalizing over-parameterized models. 

- Stepwise regression, wherein features are eliminated from an over-parameterized model in a stepwise fashon.
- Using a mathematical **regularization** technique, known as singular value decomposision, to determine the number of meaningful components for a model. 
- Using **regularizaton** methods known as ridge, lasso, and elastic-net regression to stabilize over-parameterized models. 



***
**Note:** To run the code in this note book you must have installed the following packages:
 - pandas
 - numpy
 - seaborn
 - scikit-learn
 - statsmodels
 - matplotlib
***

## Stepwise Regression

In this section we will work through an example of stepwise regression using the [Galton family height data](http://www.randomservices.org/random/data/Galton.html). The goal of the model is to predict the height of adult children given information on the height of their parents. 

<small>Francis Galton, 2017, "Galton height data", https://doi.org/10.7910/DVN/T0HSJ1, Harvard Dataverse, V1</small>

### Preparing the data

As a first step we will create a data set for just the adult male children. The code in the cell below performs the following operations:

- Subset the data to just male adult children.
- Compute two new feaures, the heights of the parents squared.
- Zscore scale the features.

Execute this code to prepare the data. 

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Load Data
data_file = 'https://library.startlearninglabs.uw.edu/DATASCI410/Datasets/galton.txt'
family_data = pd.read_csv(data_file, delimiter='\t')

# Label the columns
family_data.columns = ["family","father","mother","gender","childHeight", "kids"]

# Preview the data
family_data.head()

In [ ]:
# Check data types for numeric data types
family_data.dtypes

In [ ]:
# Subset the data with a Boolean Flag
isMale = family_data.loc[:,"gender"] == "M"

# Create just the male dataframe
male_only = family_data[isMale].copy()

# Preview the new dataframe
male_only.head()


In [ ]:
# Display counts of the data frames
print('Number of rows: {}, Number of Males: {}'.format(len(family_data), len(male_only)))

In [ ]:
# Create new data frame for new feature set
male_df = male_only.copy()

# Add in squares of mother and father heights
male_df['father_sqr'] = male_df['father'] **2
male_df['mother_sqr'] = male_df['mother'] **2

# Drop columns for family, gender, kids
Obsolete = ["family", "gender", "kids"]
for x in Obsolete:
    male_df = male_df.drop(x, axis=1)
    
# Reset the index
male_df=male_df.reset_index(drop=True)

# preview the data
male_df.head()

In [ ]:
# Scale everything but the individual height (child height)
# Create a scale function for a column in a pandas df
def scale(col):
    mean_col = np.mean(col)
    sd_col = np.std(col)
    std = (col - mean_col) / sd_col
    return std

# Add scaled x to data frame
male_df['father'] = scale(male_df['father'])
male_df['mother'] = scale(male_df['mother'])
male_df['father_sqr'] = scale(male_df['father_sqr'])
male_df['mother_sqr'] = scale(male_df['mother_sqr'])

In [ ]:
male_df.head()

### Computing a model with all features

As a first step, let's compute a model for the hight of the adult male childern using all available features. Execute the code in the cell below to compute this model, and print and plot evaluation information. 

In [ ]:
ols_model = sm.ols(formula = 'childHeight ~ father + mother + father_sqr + mother_sqr + 1', data=male_df)

results = ols_model.fit()
n_points = male_df.shape[0]
y_output = male_df['childHeight'].reshape(n_points, 1)

# Get slope (m) and y-intercept (b)
print('Intercept, Slopes : \n{}'.format(results.params))

# Get the t-values (hypothesis test statistics) for linear regression coefficient hypothesis tests.
print('Intercept t-value, Slope t-values: \n{}'.format(results.tvalues))

# Get p-values for above t-value statistics
print('\nHypothesis test summary for each coefficient if they differ from zero:')
print(results.pvalues)

print('\nSSE, SST, SSR, and RMSE:')
mean_y = np.mean(y_output)
sst = np.sum((y_output - mean_y)**2)
sse = sst - results.ssr
print('SSE: {}'.format(sse))
print('SST: {}'.format(sst))
print('SSR: {}'.format(results.ssr))
print('RMSE: {}'.format(np.sqrt(results.mse_model)))

# Get most of the linear regression statistics we are interested in:
print(results.summary())

# Plot a histogram of the residuals
sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

It is clear from the summary that this model is over-parameterized. Only the intercept is significant. In other words, we are computing the average value of the lable (childHeight), but nothing more. Examination of the residual plots shows them to be mostly well behaved, except with a bit of curvature in the standardized residual plot. 

### Apply stepwise regression

Stepwise regression using model performance metrics to prune the number of features in a model. The steps can be forward, wherein features are added one at a time in order of importance, until a point of diminished return is reached. Or, the steps can be backward, wherein a model using all features is pruned one feature at a time in reverse order of importance. It is also possible to step in both directions. In practice, either backward steps or using both directions are used, since forward steps have a tendency to get stuck at poor solutions. 

A significant issue with stepwise regression is to choose a performance metric. Many commonly used error metrics like RMSE will natually get better as we add more model parameters. Consequently the **Akaike information criterion** (AIC) is often used. We can write the AIC as:

$$AIC = 2 k - 2 ln(\hat{L})\\
where\\
\hat{L} = the\ likelihood\ given\ the\ fitted\ model\ parmaters\ \hat\theta = p(x| \hat\theta)\\
x = observed\ data\\
k = number\ of\ model\ parameters$$

In words, the AIC is the model log-likelihood adjusted for the number of model parameters. The objective is to minimize the AIC. 

The quantity $- 2 ln(p(x| \hat\theta))$ is sometimes referred to as the **deviance** of the model. Deviance is a measure of the relative likelihood of the model. Deviance is a generalization of the variance. In fact, deviance should be meaured with respect to a saturated model (number of parameters = number of observations), but this step is often skipped.

In [ ]:
def forward_selected(data, response):
    """Linear model designed by forward selection. Based on AIC

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
    """
    # Start with no factors (intercept only)
    formula = "{} ~ 1".format(response)
    best_aic = sm.ols(formula, data).fit().aic
    
    # Go through remaining sets of variables one-by-one
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_aic = best_aic
    
    # Check if any variables remain and if we haven't improved by adding any yet
    while remaining and current_aic == best_aic:
        aic_candidates = []
        for candidate in remaining:
            # Try adding the candidate column
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            # Get AIC
            aic = sm.ols(formula, data).fit().aic
            # Append tuple of the form (aic, response)
            aic_candidates.append((aic, candidate))
        # Sort all the pairs by the first entry of tuple (default of sort() method)
        aic_candidates.sort()
        # Remember that the sort() method sorts by smallest to largest of first entry here.
        #   If you were to change the criteria to something that needs to be maximized, change sort/pop order!
        best_new_aic, best_candidate = aic_candidates.pop(0)
        # Now check if we have something better:
        if best_new_aic < current_aic:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_aic = best_new_aic
        # Now we repeat the process with all the remaining candidate columns

    # Here is the final formula!
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    # Get the model object
    model = sm.ols(formula, data).fit()
    return model

model = forward_selected(male_df, 'childHeight')

print(model.model.formula)

print('Adjusted-R-Squared: {0:.3f}'.format(model.rsquared_adj))
print('AIC: {0:.3f}'.format(model.aic))

## Your Turn 1

## Backward stepwise selection

Backward stepwise selection is a very similar algorithm.  Fill in the blanks in the following formula for backward stepwise selection:

In [ ]:
def backward_selected(data, response):
    """Linear model designed by backward selection. Based on AIC

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by backward selection
    """
    # Start with all factors and intercept
    possible_factors = set(data.columns)
    possible_factors.remove(response)
    formula = "{} ~ {} + 1".format(response, ' + '.join(possible_factors))
    
    # Fill out this formula!
    #--------------------------------
    # Fill in spot #1!!!!
    best_aic = sm.ols(???).fit().aic                  # fill in spot 1, replace ???
    #--------------------------------
    
    current_aic = best_aic
    
    # Set a non-empty set of columns that will be labeled as "to remove and try"
    to_try_remove = possible_factors
    
    # Check if any variables remain and if we haven't improved by adding any yet
    while to_try_remove and current_aic == best_aic:
        aic_candidates = []
        for candidate in to_try_remove:
            
            columns = possible_factors - set([candidate])
            # Try removing the candidate column
            formula = "{} ~ {} + 1".format(response, ' + '.join(columns))
            # Get AIC
            aic = sm.ols(formula, data).fit().aic
            
            # Append tuple of the form (aic, response)
            aic_candidates.append((aic, candidate))
            
        # Sort all the pairs by the first entry of tuple (default of sort() method)
        aic_candidates.sort()
        # Remember that the sort() method sorts by smallest to largest of first entry here.
        #   If you were to change the criteria to something that needs to be maximized, change sort/pop order!
        best_new_aic, best_candidate = aic_candidates.pop(0)
        
        # Now check if we have something better:
        if best_new_aic < current_aic:
            # Remove the best candidate's name from possible_factors
            
            #--------------------------------
            # Fill in spot #2 & #3!!!!
            possible_factors.remove(???)          #fill in spot 2, replace ???
            current_aic = ???                     #fill in spot 3, replace ???
            #--------------------------------
            
        # Now we repeat the process with all the remaining candidate columns

    # Here is the final formula!
    formula = "{} ~ {} + 1".format(response, ' + '.join(possible_factors))
    # Get the model object
    model = sm.ols(formula, data).fit()
    return model

backwards_model = backward_selected(male_df, 'childHeight')

print(backwards_model.model.formula)

print('Adjusted R-Squared: {}'.format(backwards_model.rsquared_adj))
print('AIC: {}'.format(backwards_model.aic))

## Linear Model Statistics
Now that we have a backwards-selected formula : `childHeight ~ father + mother + mother_sqr + 1`, let's use that formula to get the statistics of the model.

In [ ]:
# Linear model stats
ols_model_forward = sm.ols(formula = 'childHeight ~ father + mother + mother_sqr + 1', data=male_df)

results = ols_model_forward.fit()
n_points = male_df.shape[0]
y_output = male_df['childHeight'].reshape(n_points, 1)

# Get slope (m) and y-intercept (b)
print('Intercept, Slopes : \n{}'.format(results.params))

# Get the t-values (hypothesis test statistics) for linear regression coefficient hypothesis tests.
print('Intercept t-value, Slope t-values: \n{}'.format(results.tvalues))

# Get p-values for above t-value statistics
print('\nHypothesis test summary for each coefficient if they differ from zero:')
print(results.pvalues)

print('\nSSE, SST, SSR, and RMSE:')
mean_y = np.mean(y_output)
sst = np.sum((y_output - mean_y)**2)
sse = sst - results.ssr
print('SSE: {}'.format(sse))
print('SST: {}'.format(sst))
print('SSR: {}'.format(results.ssr))
print('RMSE: {}'.format(np.sqrt(results.mse_model)))

# Get most of the linear regression statistics we are interested in:
print(results.summary())

# Plot a histogram of the residuals
sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

### Adding an interaction term

We will try one last idea, adding an interaction term. In this case we will compute all possbile interactions between the heights of the mother and the father, `mother`, `father`, and the interaction term: `mother X father`. 

Execute the code in the cell below to compute the model and print and plot the results. 

In [ ]:
# Linear model stats
ols_model_forward = sm.ols(formula = 'childHeight ~ father + mother + mother*father + 1', data=male_df)

results = ols_model_forward.fit()
n_points = male_df.shape[0]
y_output = male_df['childHeight'].reshape(n_points, 1)

# Get slope (m) and y-intercept (b)
print('Intercept, Slopes : \n{}'.format(results.params))

# Get the t-values (hypothesis test statistics) for linear regression coefficient hypothesis tests.
print('Intercept t-value, Slope t-values: \n{}'.format(results.tvalues))

# Get p-values for above t-value statistics
print('\nHypothesis test summary for each coefficient if they differ from zero:')
print(results.pvalues)

print('\nSSE, SST, SSR, and RMSE:')
mean_y = np.mean(y_output)
sst = np.sum((y_output - mean_y)**2)
sse = sst - results.ssr
print('SSE: {}'.format(sse))
print('SST: {}'.format(sst))
print('SSR: {}'.format(results.ssr))
print('RMSE: {}'.format(np.sqrt(results.mse_model)))

# Get most of the linear regression statistics we are interested in:
print(results.summary())

# Plot a histogram of the residuals
sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

It is clear from the summary that the interaction term is not significant. The best model we have to this point is the one computed with stepwise regression. 

***
**Note:** Stepwise regression appears to be a simple method for feature selection. However, be aware that **stepwise regresson does not scale well**. As with any multiple comparison method, stepwise regresson suffers from a high probability of false positive results. In this case, a feature which should be dropped might not be, because of a low p-value or AIC. 
***

## Regularization: Singular Value Decomposition

Now that we have explored both manual feature selection and stepwise regression, we will examine regularization methods. Regularization methods stabilize the inverse of the **model matrix**. In this section we will use the singular value decomposition method to stabilize a model matrix. 

You may well wonder why we need regularization methods, when we have tools like stepwise regression. Two important reasonse are:

- Stepwise regression is a compuationally intensive process, since we must recompute the model many times. There are methods that allow computation of the updated model, but with a large number of features there are numerous permutations. We need methods that can handle hundreds, thousands, or even millions of features.

> Note: Consider a small data set with only 20 features.  The amount of possible linear models with NO interaction terms is given by:  $\binom{20}{1} + \binom{20}{2} + \binom{20}{3} + \dots + \binom{20}{20}$, which is the sum of the 21st row of the Pascal's triangle.  This comes out to be $2^{20} = 1,048,576$.  Although, according to our algorithm, this would be the maximum number of models to compute, you can see how computationally hard this becomes.

- With stepwise regression a feature is either in or out of the model. This may not be the best choise. Perhaps a re-weighing the features in some way might be better. 
- Stepwise regression suffers from issues inherent in multiple comparisons. 

In order to understand the motivation and methods of feature selection/transformation, we will start with reviewing some linear algebra.

### Linear Algebra Review

Before we get into the details of regularizaton, let's review some basic linear algebra.

For this part of the notebook we will be using the library `numpy`, which is the numerical python library.

> Note: In python there are a few distinctions to make.  The base python has a variable type called 'Lists'. Lists are just that. They are lists of objects. `[5, 'foobar', True, ...]`. While we can make a list that contains all numbers: `[2.0, 2.0, 2.0]`, this is not a vector nor matrix.  The package `numpy` introduces arrays (vectors and matrices) which have the correct associated matrix properties.

> Note: Please be careful using Jupyter Notebook. Jupyter displays arrays the same as lists (you'll see in examples below).  It is even the case that some Numpy functions and objects will interact with lists the same way, but sometimes they do not and it can be frustrating to troubleshoot if they appear the same.  Always read documentation and check the types of your variables with the command `type(my_variable)`.

Let's start by creating two vectors of length 3.

In [ ]:
a_list = [2]*3
print(a_list)
print(type(a_list))
a = np.array([2]*3)
print(a)
print(type(a))
b = np.arange(1, 4)
print(b)
print(type(b))

### Element-wise Operations
We can perform some basic element-wise arithmetic opertions on vectors.

In [ ]:
print(a + b)
print(a * b)

These vectors are both dimension (size) three.

We can do the same in two - dimensional matrices as follows:

In [ ]:
a2d = np.array([[2]*3, [3]*3])
print('a: \n{} \n a-size: {}'.format(a2d, a2d.shape))
b2d = np.reshape(np.arange(1,7), newshape=(2, 3))
print('\nb: \n{} \n b-size: {}'.format(b2d, b2d.shape))

# Addition
print('\nAddition: \n {}'.format(a2d + b2d))
# Multiplication
print('\nMultiplication: \n {}'.format(a2d * b2d))

### Transposing a Matrix
We can also _transpose_ a two-dimensional matrix by flipping the rows and columns. We use the `numpy` method `np.transpose`.

In [ ]:
print('Try transposing on a vector...')
print(np.transpose(a))
# Uh oh!  Transpose on a vector doesn't do anything...

print('Now reshape a into 2D and then transpose:')
print(np.transpose(np.reshape(a, newshape=(1,3))))
print('Now transpose a 2d matrix:')
print(np.transpose(a2d))

### Matrix Dot Product
We can also compute the **dot product**, which is also known as the **scalar product** or **inner product** of two vectors of equal length.

$$dot\ product = \Sigma_i^n a_i \cdot b_i$$

Give the dot product a try by executing the code in the cell below. We use the `numpy` method `np.dot`.

In [ ]:
print('np.dot(a, b) = np.dot({}, {})'.format(a, b))
np.dot(a, b)

The square root of the inner product of a vector with itself is the length or $L2$ norm of the vector.

$$\parallel a \parallel = length\ of\ vector\ a = \sqrt{a \cdot a}$$

--------------------

## Your Turn 2

Create and exectue the code to compute the length or norm of the vector `a` in the cell below.

In [ ]:
def inner_prod(vec1, vec2):
    # Replace the ??? in the following calculation:
    inner_prod_calc = ???
    return inner_prod_calc

def l2_norm(my_vec):
    # Replace the ??? in the following calculation:
    l2_norm_calc = ???
    return l2_norm_calc

print('L2 Norm of {} = {}'.format(a, l2_norm(a)))
# Should get ~3.4641016151377544

We can also write the inner product as:

$$a \cdot b = \parallel a \parallel \parallel b \parallel cos(\theta)\\
or \\
cos(\theta) = \frac{a \cdot b}{\parallel a \parallel \parallel b \parallel}$$

Notice that the inner product of orthogonal vectors is $0$. Run the code in the cell below to see an example.

In [ ]:
aa = np.array([1, 0, 0])
bb = np.array([0, 1, 1])
print(inner_prod(aa, bb))

Let's try some operations on martricies. Let $A$  and $B$ be $m = 4$ rows by $n = 3$ columns matrices. 

In [ ]:
A = np.array([[4]*3]*4) # A has 4 rows and 3 columns
print(A)
B = np.array(np.reshape(np.arange(1, 13), newshape = (4, 3)))
print(B)

We can perform some arithmatic operations element by element on these matrces.

In [ ]:
print(A + B)
print(A * B)

We can multiply a $mxn$ matrix by a vector of length $n$ by taking the inner product of each row of the matrix and the vector. The result in a vector of length $n$. Each element of the result can be written at:

$$y_i = \Sigma_j^m A_{ij} \cdot b_j$$

Run the code in the cell below and examine the result. 

In [ ]:
np.dot(A, b)

So, how do we multipy two matrices? In matrix multiplcation each element of the resulting matrix is the inner product of a row by a column. For example, the element $Y_{ij}$ of the result matrix is computed as follows:

$$Y_{ij} = \Sigma_j^m A_{ij} \cdot B_{ji}$$

Notice that the number of columns, $m$, of the first matrix must equal the number of rows of second matrix. And, that the number of rows, $n$ of the first matrix must equal the number of columns of the second matrix. In this case, the two matrices are said to be **conformable**. 

Give matrix multiplication a try by exectuing the code in the cell bellow.

In [ ]:
np.dot(A, B)

That operation failed! Evidently these matrices are not conformable. 

But, what if we take the transpose of $B$? The **transpose** of a matrix is just that matrix with the row and column indicies permuted like this:

$$B_{ji}^T = B_{ij}\\
where \\
B\ has\ dimensions\ n x m \\
and \\
B^T\ has\ dimensions\ m x n$$ 

If we multiply an $n x m$ matix by an $m x n$ matrix the result is a square $n x n$ matrix. 

We can execute the code in the cell below to multiply the matrix A by the transpose of B. Use the np.transpose() function to take the transpose of the matrix:

In [ ]:
AtB = np.dot(np.transpose(A), B)
print('A\'*B = \n{}'.format(AtB))
print('Shape of A\'*B = {}'.format(AtB.shape))

ABt = np.dot(A, np.transpose(B))
print('\nA*B\' = \n{}'.format(ABt))
print('Shape of A*B\' = {}'.format(ABt.shape))

### Identity Matrix
We can define the **identity** matrix having ones on the diagonal and zeros elsewehere.

$$I = \begin{bmatrix}
    1  & 0 & 0 & \dots & 0 \\
    0  & 1 & 0 & \dots & 0 \\
    \vdots &\vdots &\vdots & & \vdots \\
    0 & 0 & 0 & \dots & 1
\end{bmatrix}$$

The identity multiplied by any matrix gives that matrix. If $AB$ is a rectangular matrix then:

$$AB = I \cdot AB = AB \cdot I$$

In `numpy` the Identity matrix is called `np.eye` and takes as an argument the number of rows/columns.

In [ ]:
I3 = np.eye(3)
I4 = np.eye(4)

print('I (3X3) = \n{}'.format(I3))

print(np.dot(I3, AtB))
print(np.dot(I4, ABt))

### Inverse of a Matrix
In principle we can compute an inverse of a matrix so that:

$$A = A\\
A = AI \\
A^{-1}A = I$$

Use the Linear Algebra methods in `numpy` to get the inverse of matrix $M$ with `np.linalg.inv`.

In [ ]:
M = np.array([[1., 4.], [-3., 2.]])
M_inverse = np.linalg.inv(M)

print('M: \n{}'.format(M))

print('\nM_inv = \n{}'.format(M_inverse))

print('\nM_inv * M = \n{}'.format(np.dot(M_inverse, M)))

print('\nM * M_inv = \n{}'.format(np.dot(M, M_inverse)))

## Singular Value Decomposition
In machine learning, we often encounter matricies which cannot be inverted directrly. Instead, we need a decompositon of $A$ that allows us to compute $A^{-1}$. One possibility is a method called singular value decomposition or SVD:

$$svd(A) = U D V^{\ast}$$

Where,

- $U$ are the orthogonal unit norm left singular vectors.
- $V$ are the orthogonal unit norm right singular vectors, and $V^{\ast}$ is the conjugate transpose. For real-valued $A$ this is just $V^T$.
- $D$ is a diagonal matrix of singular values, which are said to define a **spectrum**.
- $A$ is comprised of the linear combination of singular vectors scaled by singular values.

To compute the SVD of a matrix and view the results execute the code in the cell below. We use the `numpy` method called `np.linalg.svd` using `s` for the singular values.

In [ ]:
U, s, V = np.linalg.svd(B, full_matrices=False)
print('U: {}'.format(U))
print('s: {}'.format(s))
print('V: {}'.format(V))

In [ ]:
# It's called a factorization because...
# Create a diagonal matrix
S = np.diag(s)

# Matrix multiply:  U * S * V
B_reconstruction = np.dot(U, np.dot(S, V))
np.allclose(B, B_reconstruction)

----
Hint: check the values of S, B, and B_reconstruction
<hr>

You can verify that the singular vectors form a orthonomal basis by checking to see if $U \cdot U^{T} = U^{T} \cdot U = I$ and $V \cdot V^{T} = V^{T} \cdot V = I$. We can check this by executing the code in the cell below:

In [ ]:
basis_vec_check1 = np.dot(np.transpose(U), U)
print(np.round(basis_vec_check1, 2))

basis_vec_check2 = np.dot(V, np.transpose(V))
print(np.round(basis_vec_check2, 2))

The result are two identity matricies!

We can view the product of the matrix $A$ with a vector as defining a rotation and scaling. The singular value decomposition of $A$ can be viewed as:

- A first rotation defined by the unit norm singular values $V^{\ast}$.
- A scaling defined by the diagonal singular value matrix $D$.
- A second rotation defined by the unit norm singular values $U$.

This geometric interpertation can be visualized as shown in the figure below.

<img src="https://library.startlearninglabs.uw.edu/DATASCI410/img/SVD.png" title="Singular Value Decomposition" style="height: 300px;">

Execute the code in the cell below and examine the rotations and scaling of the inital vector.

In [ ]:
# 2-Dimensional Example
D = np.array([[3., 4.], [1., 5.]])
U, s, V = np.linalg.svd(D, full_matrices=False)
e = np.sqrt(1./2.)
u = np.array([e, e])

print('u: {}'.format(u))
print('Magnitude: {}'.format(np.sqrt(np.dot(np.transpose(u), u))))
print('\n')

print('First Rotation:')
u_rot1 = np.dot(np.transpose(V), u)
print(u_rot1)
print('Magnitude: {}'.format(np.sqrt(np.dot(np.transpose(u_rot1), u_rot1))))
print('\n')

print('Scaling:')
u_scaled = np.dot(np.diag(s), u_rot1)
print(u_scaled)
print('Magnitude: {}'.format(np.sqrt(np.dot(np.transpose(u_scaled), u_scaled))))
print('\n')

print('Second Rotation:')
u_rot2 = np.dot(U, u_scaled)
print(u_rot2)
print('Magnitude: {}'.format(np.sqrt(np.dot(np.transpose(u_rot2), u_rot2))))

# ---------------
# Check with just D * u
print('\n')
print('Originally, D * u = {}'.format(np.dot(D, u)))

## Your Turn 3:
Compute the SVD of matrix $B$ and do the following:
- Demonstrate that the 4X3 matrix $B$ gives the same rotation and scale by multipling a vector `u = np.array([e, e, e])` where `e = np.sqrt(1./3.)` as multiplying the decomposition, $U \cdot S \cdot V' = B$  by `u`.

In [ ]:
# Get SVD decomposition
U, s, V = np.linalg.svd(B, full_matrices=False)
S = np.diag(s)

# Setup a 3-D rotational vector.
e = np.sqrt(1./3.)
u = np.array([e, e, e])

# Replace the ???
# Calculate Rotation:
# u_rot = V' * u
u_rot = ???
print('\nRotation: {}'.format(u_rot))

# Calculate Scaling:
# u_scaled = diag(s) * u_rot
u_scaled = ???
print('\nScaled: {}'.format(u_scaled))

# Calculate the last rotation:
# u_rot2 = U * u_scaled
u_rot2 = ???
print('\nRotation 2: {}'.format(u_rot2))

#------ Are they the same? Caluclate just B*u --------

# Calculate r = B * u:
r = ???
print('\nr = B * u = {}'.format(r))

## Pseudo Inverse
We will be interested in the inverse of a matrix A to solve linear regression.  If a matrix, $A$ has a SVD decomposition, $U \cdot S \cdot V'$, then the inverse, $A^{-1}$ can be calculated as follows:

$$ A^{-1} = \left( U \cdot S \cdot V'  \right)^{-1} $$

$$ = V'^{-1} \cdot S^{-1} \cdot U^{-1} $$

From before, we know (and you can verify with numpy operations) that the inverse of the matrices $V$ and $U$ are transposes. I.e, $U^{-1} = U'$ and $V^{-1}=V'$.  Also, remember that $S$ is a diagonal matrix, with zero values off the diagonal. The inverse of $S$ is easy to calculate:

If,

$$
S = \textrm{diag}(s_{1}, s_{2}, ...)
$$

Then,

$$
S^{-1} = \textrm{diag}\left( \frac{1}{s_{1}}, \frac{1}{s_{2}} , ... \right)
$$

We end up with:

$$ A^{-1} = V \cdot S^{-1} \cdot U' $$

This representation of $A^{-1}$ is called the **pseudo inverse** also known as the **Moore-Penrose inverse**.

It is commonly written as:

$$A^\dagger = V \cdot D^+ \cdot U'$$

Where,

- $D^+$ is the square diagonal matrix of **inverse singular values** significantly greater than $0$. All other terms are set to $0$.
- $U'$ is the transpose of the right sigular value matrix. 
- $V$ is the left singular value matrix.

The matrix, $A$ may not be of full rank. The types of long and narrow $n x m$ matricies we encounter in machine learning are typically **rank deficient**. A rank deficient matrix arises when there is linear dependency between one or more of the columns. As an example, a matrix with correlated (not necessiarily perfectly correlated) columns is bound to be rank deficient. 

A matrix is considered rank deficient if it has one or more of the $m$ singular values  $d_i  \sim 0.0$. In this case we substitute $0.0$ values on the diagonal of $D^+$ where the singular values $d_i \sim 0.0$. In fact, we want $d_i$ to be significanlty greater than $0$. 

Let's try an example. The code in the cell below computes the SVD of a matrix of random numbers chosen from a Normal distribution. The pseudo inverse is computed and multiplied by the original matrix. Execute this code and note the result.

In [ ]:
print('Create a matrix of random normal values:')
C = np.array(np.random.randn(3, 3))
print(C)

print('\nCompute the SVD and look at the singular values:')
U, s, V = np.linalg.svd(C, full_matrices=False)
print(s)

print('\nThe inverse matix of singular values:')
D = np.diag(np.reciprocal(s))
print(D)

print('\nThe Pseudo-inverse of the matrix:')
cInv = np.dot(np.dot(np.transpose(V), D), np.transpose(U))
print(cInv)

print('\bThe Pseudo-inverse times the matrix:')
out = np.dot(cInv, C)
print(np.round(out, 2))

First, notice that the singular values are of similar magnitude and none are near zero. This matrix is not rank deficient. 

### Pseudo Inverse Example
Let's try another example. The code in the cell below does the following:

- Creates a $4 X 4$ matrix of numbers drawn from a Normal distribution.
- Substitues values in the 4th column which are a linear combination of the other three columns.
- Computes the SVD of this matrix.
- Creates the inverse diagonal matrix of singular values.
- Computes the pseudo inverse of the matrix.
- Multiplies the pseudo inverse by the original matrix. 

Execute this code and examine the result.

In [ ]:
print('Create a matrix of random normal values:')
C = np.array(np.random.randn(4, 4))
# Rewrite the 4th column as a linear combination of the prior 3 columns
C[:, 3] = 0.4 * C[:, 0] + 0.2 * C[:, 1] + 0.4 * C[:, 2]
print(C)

print('\nCompute the SVD and look at the singular values:')
U, s, V = np.linalg.svd(C, full_matrices=False)
print(s)

print('\nThe inverse matix of singular values:')
D = np.diag(np.reciprocal(s))
print(D)

print('\nThe Pseudo-inverse of the matrix:')
cInv = np.dot(np.dot(np.transpose(V), D), np.transpose(U))
print(cInv)

print('\bThe Pseudo-inverse times the matrix:')
out = np.dot(cInv, C)
print(np.round(out, 2))

Notice the following about this result:

- The 4th singular value is nearly zero. Evidently, this matrix is rank deficient. 
- The product of the pseudo inverse is not close to being the identity matrix. This is the result of taking using the unstable inverse of the rank deficient matrix.

Instead, let's set the singular value that is near zero to actually zero.

- Set the inverse of the smallest singular value to zero.
- Compute the pseudo inverse.
- Compute the product of the pseudo inverse and the orginal matrix. 
- Note if the result is closer to an identity matrix. 

In [ ]:
print('The inverse matrix of singular values:')
print(D)

# Fill in the missing spots
print('\nSet the near-zero singular values to zero:')
D[3, 3] = 0.0
print(D)

print('\nThe pseudo-inverse of the matrix:')
cInv = np.dot(np.dot(np.transpose(V), D), np.transpose(U))
print(cInv)

print('\nThe pseudo-inverse times the matrix:')
out = np.dot(cInv, C)
print(out)

### Regression with the Pseudo Inverse

We have already looked at feature selection using manual trial and error methods and stepwise regression. How can we use the pseudo inverse to create a regularized regression? 

Let's start by examining the linear regression problem. The goal is to compute a vector of **model coefficients** or weights which minimize the mean squared residuals, given a vector of data $x$ amd a **model matrix** $A$. We can write our model as:

$$x = A b$$

To solve this problem we would ideally like to compute:

$$b = A^{-1}x$$

However, this is hard to do directly in practice.

- In typical case $A$ is long and narrow. In other words we have more data **cases** than coeficients. 
- Solving for $A^{-1}$ is computationally difficult and inefficient.
- Solution is numerically unstable if $A$ is rank deficient. 

One way to deal with the problem of rank deficiency is to use the pseudo inverse $A^\dagger$. Recalling the singular value decomposition of $A$ we can write:

$$A^\dagger = V D^+ U^*$$

***
**Note:** In practice, the direct compuation of a pseudo inverse is rarely used for linear models. Instead, more compuationally efficient methods such as QR decomposition are often used. Discussion of these methods is beyond the scope of this course. Details can be found in many sources including the seminal book titled [Matrix Computations](http://web.mit.edu/ehliu/Public/sclark/Golub%20G.H.,%20Van%20Loan%20C.F.-%20Matrix%20Computations.pdf) by Gene Golub and Charles van Loan.
***

As a first step we need to create a model matrix of the features for the Galton height data. Execute the code in the cell below which creates a matrix from the features in a data frame.

In [ ]:
male_df_vars = male_df.loc[:,['mother', 'father','mother_sqr', 'father_sqr']]
M = male_df_vars.as_matrix()
childHeight = male_df.loc[:,'childHeight']

print('1st 5 rows of Data Frame: ')
print(male_df_vars.head())
print('\n1st 5 rows in matrix Form: ')
print(M[0:5, :])

The next step is to compute the SVD of the resulting matrix and examine the singular values. In the cell below, execute code to do the following:
1. We will use Numpy's built in function, `np.linalg.lstsq` to perform linear regression via the SVD.
2. We extract the beta coefficients.
3. From these coefficients, we perform the predictions.
4. We will also compute the R-squared.

In [ ]:
beta_coeffs, resids, rank, s = np.linalg.lstsq(M,childHeight)

print('beta coefficients: {}'.format(beta_coeffs))

print('\nsingular values: {}'.format(s))

Notice that the 3rd and 4th singular values are a few orders of magnitude smaller than the first two. This matrix is most likely rank deficient.

### Model Evaluation
Now that we have a vector of model coefficients, its time to evaluate our model.  We use the following steps:

- Compute the predicted values or scores using the product of the model matrix and the model coefficients. We need to add the mean of the label values.
- Compute the residuals.
- Display residual plots and summary statistics.

Execute this code and examine the results. 

In [ ]:
male_predictions = np.dot(M, beta_coeffs) + np.mean(childHeight)
male_resids = male_predictions - childHeight

# Plot the residuals vs score (height)
plt.subplot(1, 2, 1)
plt.plot(male_predictions, male_resids, '.')
plt.ylabel('residuals')
plt.xlabel('score(height)')
plt.title('Residuals vs Score')

# Histogram of residuals
plt.subplot(1, 2, 2)
plt.hist(male_resids)
plt.title('Histogram of Residuals')

# R-squared
#              SSR
# R^2 = 1 -  -------
#              SST

SSR = np.sum(np.square(male_resids))
SST = np.sum(np.square(childHeight - np.mean(childHeight)))

print('SSR: {}'.format(SSR))
print('SST: {}'.format(SST))

R2 = 1.0 - (SSR / SST)

print('R-squared = {}'.format(R2))


## Principal Component Regression

In various regression transformations we have applied, we haven't yet asked ourselves, "What is a good transformation to the independent features that may create an optimal amount of independence between them?"

Here, the idea is to combine the features into new wholly independent features by combining them in various linear fashions.

Imagine we are performing linear regression on multiple features (n of them), where our model is as follows:

$$
y_{i} = \beta_{0} + \beta_{1}x_{1} + ... + \beta_{n}x_{n} + \epsilon_{i}
$$

We might just procede forward and assume that our x-features are independent and see how our model performs.  What if we could find new features such that they are guarenteed to be completely independent of each other as follows:

$$
y_{i} = \beta_{0} + \beta_{1}\cdot f_{1} \left( x_{1}, ... , x_{n} \right) + ... + \beta_{n}f_{n} \left( x_{1}, ... , x_{n} \right) + \epsilon_{i}
$$

Here, we are creating new features, which are linear combinations of all our prior x-features in our $f()$ functions.  If we write out what an $f()$ function might look like:

$$
f_{m} = a_{0} + a_{1} \cdot x_{1} + ... + a_{n} \cdot x_{n}
$$

The question still remains, how do we find such magical $f()$ functions that assures us that the new features, $f_{m}$ taken together are 100% independent of each other?

The method of finding these very important components is called Principal Component Analysis (or PCA).  Then when we use these component functions to do regression, it is called Principal Component Regression (or PCR).

It is considered a regularized regression because, typically, we end up with less features than we started with because, in reality, there is always some dependence in our data.

You can imagine that, depending on how much dependence is in our data, that the first components found will explain a majority of the variance in the response and the last components add very little explanatory power.  It is common to choose a smaller subset of PCA regressors to do PCR.

The benefits of PCR is that it assures us that our new features are completely independent, which overcomes the multicollinearity problem in regression.

The downside is that the new features, being functions of variables, are less interpretable than using the original features.

-----------------------
PCR has these steps (commonly):

--------------------------

0. Scale the input features in your data matrix!

1. Perform PCA on the observed data matrix to obtain the principal components

2. Select a subset of the principal components for regression. Usually we select the components that explain the most of the variance in the dependent variable.

3. Perform linear regression using the selected principal components as features


The reason we cover this regression method after introducing singular value decomposition is that the SVD is used to find the principal components of the data matrix.

For exploring this method, we use generated data to illustrate how components work on features that are not fully independent.

To compute PCR, we will use the PCA function from the `sklearn decomposition` library.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

np.random.seed(10)

# We right-multiply a 2x2 matrix to a 2x100 matrix to transform the points to be related.
X = np.dot(np.random.randn(100, 2), np.random.rand(2, 2))
# Scale the points
X = StandardScaler().fit_transform(X)

plt.scatter(X[:, 0], X[:, 1])
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.title('Generated x-y data with dependence')

Let's look at the first two principal components, because we can visualize 2 dimensions in a graph.

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X)
pca_df = pd.DataFrame(data = pca_result , columns = ['pc1', 'pc2'])

print(pca_df.head())
print(pca_df.shape)

Plot the output!

In [ ]:
plt.scatter(pca_df.loc[:, 'pc1'], pca_df.loc[:, 'pc2'])
plt.ylabel('Component 2')
plt.xlabel('Component 1')
plt.title('Two Dimensional PCA')
plt.grid()

We can also look at how much variance _in the original features_ we are explaining with each principal component.

In [ ]:
print(pca.explained_variance_)
print(pca.components_)

How do we interpret the above variances and components?

For the "explained variance" we interpret those values as the variance of the dataset when all the data is projected onto that $f()$ function axis.  So here, the first $f()$ function (or principal component) explains 1.95 units of the variance. The second component explains the remaining 0.05 units of variance.

For the components, this tells us the direction of the principal components. In fact, all principal components (i.e. rows) in this matrix have the property that the magnitude of them is exactly 1.  Here, $\sqrt{(-0.707...)^{2} + (-0.707...)^{2}} = 1$.

The combination of these two matrices (explained variance and component direction) tells us that we can plot these vectors amongst the original data to see them.

In [ ]:
exp_var = pca.explained_variance_
components = pca.components_

# Vector to plot
#    = plot scale * component direction * component length
v1 = 2 * components[0] * np.sqrt(exp_var[0])
v2 = 2 * components[1] * np.sqrt(exp_var[1])
c = (0, 0) # Center is at 0,0 because of our standardization

plt.scatter(X[:, 0], X[:, 1], alpha=0.5)
plt.annotate('', c + v1, c, arrowprops={'arrowstyle': '->', 'shrinkA': 0,
                                        'shrinkB': 0, 'linewidth': 3})
plt.annotate('', c + v2, c, arrowprops={'arrowstyle': '->', 'shrinkA': 0,
                                        'shrinkB': 0, 'linewidth': 3})
plt.axis('equal')
plt.grid()
plt.title('F-vector functions (Principal Components)')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### What now?


From the above plot, we can see that the two vectors we've plotted are completely independent of each other. In other words, they are perpindicular!

We can extend this to higher dimensional data and it has many uses:

- If we have very high dimensions, we can reduce the number of dimensions needed for a good regression fit.

- Calculating the Principal Components removes any multi-collinearity issues in the data.

- We can project the original data points onto our principal component axes and perform regression.

We will combine everything we've learned into a principal component linear regression with actual data below. (Galton Height Data for females this time)

## Your Turn 4:

If our Galton height data set has 4 features, how many principal components will there be?


Let us compute all the principal components for our Galton height data on the female data points.

In [ ]:
# Subset the data with a Boolean Flag
isFemale = family_data.loc[:,"gender"] == "F"

# Create just the female dataframe
female_data = family_data[isFemale].copy()

In [ ]:
# Create new data frame for new feature set
female_df = female_data.copy()

# Add in squares of mother and father heights
female_df['father_sqr'] = female_df['father'] **2
female_df['mother_sqr'] = female_df['mother'] **2

# Drop columns for family, gender, kids
Obsolete = ["family", "gender", "kids"]
for x in Obsolete:
    female_df = female_df.drop(x, axis=1)
    
# Reset the index
female_df=female_df.reset_index(drop=True)

# Add scaled x to data frame
female_df['father'] = scale(female_df['father'])
female_df['mother'] = scale(female_df['mother'])
female_df['father_sqr'] = scale(female_df['father_sqr'])
female_df['mother_sqr'] = scale(female_df['mother_sqr'])

# preview the data
female_df.head()

In [ ]:
# Calculate all the Principal components
X = female_df.as_matrix(columns=['father', 'mother', 'father_sqr', 'mother_sqr'])
y = female_df['childHeight']
pca = PCA()
pca_result = pca.fit_transform(X)
pca_df = pd.DataFrame(data = pca_result, columns=['pc1', 'pc2', 'pc3', 'pc4'])

# Here is our data projected onto the four principal components.
print(pca_df.head())
print(pca_df.shape)

pca_df['childHeight'] = female_df['childHeight']

Now that we have all our data projected on our 4 total principal components, let's just look at the explained variance:

In [ ]:
pcr_model = sm.ols(formula = 'childHeight ~ pc1 + pc2 + pc3 + pc4', data=pca_df)

results = pcr_model.fit()
n_points = pca_df.shape[0]
y_output = pca_df['childHeight'].values.reshape(n_points, 1)

# Get most of the linear regression statistics we are interested in:
print(results.summary())

# Plot a histogram of the residuals
sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

Here, you may see that the last two principal components are not needed for a significant fit.  You could also have noticed that from the 'explained variance' for each component- and how the last two components explained nearly 4 orders of magnitude less variance.

## Regularization with Ridge and Lasso Regression

So far, we have looked at two approached for dealing with over-parameterized models; feature selection by stepwise regresson and singular value decomposision. In this section we will explore the most widely used regularization method for optimization-based machine learning models, **ridge regression**. 

Let's start by examining the matrix-equation formulation of the linear regression problem. The goal is to compute a vector of **model coefficients** or weights which minimize the mean squared residuals, given a vector of data $x$ and a **model matrix** $A$. We can write our model as:

$$x = A b$$

To solve this problem we would ideally like to compute:

$$b = A^{-1}x$$

Rewriting this equation,

$$b = A^{-1} (A^{T})^{-1} A^{T}x$$

Or, alternatively,

$$b = (A^TA)^{-1}A^Tx$$

Now, $A^TA$ is an $m x m$ matrix, and thus is of reduced dimension. But, **$A^TA$ can still be rank deficient!** 

The basic idea of ridge regression is to stabilize the inverse singular value matrix, $D^+=A^{T}A$, by **adding a small bias term**, $\lambda$, to each of the singular values. We can state this operation in matrix notation as follows. We start with a modified form of the normal equations (also know as the **L2 or Euclidean norm** minimization problem):

$$min [\parallel A \cdot x - b \parallel + \parallel \lambda \cdot b\parallel]\\  or \\
b = (A^TA + \lambda^2)^{-1}A^Tx$$

In this way, the values of small singular values do not blow up when we compute the inverse. You can see this by writing out the $D^+$ matrix with the bias term.

$$D_{ridge}^+  = \begin{bmatrix}
    \frac{1}{d_1 + \lambda^2}  & 0 & 0 & \dots & 0 \\
    0  & \frac{1}{d_2 + \lambda^2} & 0 & \dots & 0 \\
    \vdots &\vdots &\vdots & & \vdots \\
    0 & 0 & 0 & \dots & \frac{1}{d_m + \lambda^2}
\end{bmatrix}$$

Adding this bias term creates a 'ridge' in the singular value matrix, giving this method its name **ridge regression**. 

You can also think of ridge regression as limiting the L2 or Euclidean norm of the values of the model coefficient vector. The value of $\lambda$ determines how much the norm of the coefficient vector constrains the solution. You can see a view of this geometric interpretaton in the figure below.  

<img src="https://library.startlearninglabs.uw.edu/DATASCI410/img/L2.jpg" title="Geometric View of L2 regularization" style="height: 300px;">
<center> **Geometric view of L2 regularization**

The same method goes by some other names, as it seems to have been 'invented' several times. In particular, **Tikhonov regularization**, or **L2 norm regularization**. In all likelihood the method was first developted by the Russian mathematician Andrey Tikhonov in the late 1940's, and first published in English in 1977.

Let's give this a try. Execute the code in the cell below which computes the $(A^TA + \lambda^2)^{-1}A^T$ matrix with a lambda value of `0.1`.

In [ ]:
U, s, V = np.linalg.svd(M, full_matrices=False)

# Calculate the inverse singular value matrix from SVD
lambda_val = 1.0
d = np.diag(1. /  (s + lambda_val))

print('Inverse Singular Value Matrix:')
print(d)

# Compute pseudo-inverse
mInv = np.dot(np.transpose(V), np.dot(d, np.transpose(U)))

print('M Inverse')
print(mInv)

Now we compute the model coeficients using the $(A^TA + \lambda^2)^{-1}A^T$ matrix we just computed.

In [ ]:
beta_coeffs_ridge = np.dot(mInv, childHeight)
print('Beta\'s: {}'.format(beta_coeffs_ridge))

### Bias-variance trade-off

The statsmodels package allows us to compute a sequence of ridge regression solutions.  The function that does this uses a method called 'elastic-net', know that ridge regression is a specific case of elastic-net, and we will talk more about this later.

The code in the cell below computes solutions for `20` values of $\lambda$. Execute this code and examine the values of the model coefficients as $\lambda$ increases. We'll use the `numpy` method `np.linspace` to compute sequence of lambdas.

In [ ]:
# Ridge Regression with various penalties in Statsmodels
# Generate a sequence of lambdas
log_lambda_seq = np.linspace(-6, 2, 50)
lambda_seq = np.exp(log_lambda_seq)

coeffs_array = []
rsq_array = []
formula = 'childHeight ~ mother + father + mother_sqr + father_sqr + 1'

for lamb in lambda_seq:
    ridge_model = sm.ols(formula, data=male_df).fit_regularized(method='elastic_net', alpha=lamb, L1_wt=0)
    coeffs_array.append(list(ridge_model.params))
    predictions = ridge_model.fittedvalues
    residuals = [x - y for x, y in zip(np.squeeze(predictions), childHeight)]

    SSR = np.sum(np.square(residuals))
    SST = np.sum(np.square(childHeight - np.mean(childHeight)))

    rsq = 1 - (SSR / SST)
    rsq_array.append(rsq)

In [ ]:
# Pull out partial slopes (drop intercept version)
beta_coeffs = [x[1:] for x in coeffs_array]
plt.plot(log_lambda_seq, beta_coeffs)
plt.title('Partial Slopes vs Log-Lambda')
plt.ylabel('Partial Slope Values')
plt.xlabel('Log-Lambda')

In [ ]:
# Plot partial slopes vs R squared (% deviance explained)
plt.plot(rsq_array, beta_coeffs)
plt.xlim([0.0, 0.25])
plt.title('Partial Slopes vs Log-Lambda')
plt.xlabel('R-squared')
plt.ylabel('Partial Slopes')

> Notice that $\lambda$ increases, the values of the 4 model coefficients decrease toward zero. When all coefficients are zero, the model predicts all values of the label as zero! In other words, high values of $\lambda$ give highly biased soluions, but with very low variance. For small values of $\lambda$, the situation is just the opposite. The solution has low bias, but is quite unstable, having maximum variance. This **bias-variance trade off** is a key concept in machine learning.

> Also note how we can get a significant increase in R^2 (from 0 --> ~0.20) without changing the partial slopes much.



----------------------------
### Lasso regression
----------------------------

We can also do regularization using other norms. **Lasso or L1 regularizaton** limits the sum of the absolute values of the model coefients. The L1 norm is sometime know as the **Manhattan norm**, since distrance are measured as if you were traveling on a rectangular grid of streets. 

You can also think of lasso regression as limiting the L1 norm of the values of the model coefficient vector. The value of $\lambda$ determines how much the norm of the coefficient vector constrains the solution. You can see a view of this geometric interpretaton in the figure below.  

<img src="https://library.startlearninglabs.uw.edu/DATASCI410/img/L1.jpg" title="Geometric view of L1 regularization" style="height: 300px;">
<center> **Geometric view of L1 regularization**

By setting the `alpha` argument to the statsmodels functions to zero you can perform lasso regresson. Execute the code in the cell below to compute and evaluate a lasso regression model with 20 values of lambda.

In [ ]:
# Lasso Regression with a sequence of lambdas
# Generate a sequence of lambdas
log_lambda_seq = np.linspace(-6, 2, 50)
lambda_seq = np.exp(log_lambda_seq)

coeffs_array = []
rsq_array = []
formula = 'childHeight ~ mother + father + mother_sqr + father_sqr + 1'

for lamb in lambda_seq:
    ridge_model = sm.ols(formula, data=male_df).fit_regularized(method='elastic_net', alpha=lamb, L1_wt=1)
    coeffs_array.append(list(ridge_model.params))
    predictions = ridge_model.fittedvalues
    residuals = [x - y for x, y in zip(np.squeeze(predictions), childHeight)]

    SSR = np.sum(np.square(residuals))
    SST = np.sum(np.square(childHeight - np.mean(childHeight)))

    rsq = 1 - (SSR / SST)
    rsq_array.append(rsq)

In [ ]:
# Pull out partial slopes (drop intercept version)
beta_coeffs = [x[1:] for x in coeffs_array]
plt.plot(log_lambda_seq, beta_coeffs)
plt.title('Partial Slopes vs Log-Lambda')
plt.ylabel('Partial Slope Values')
plt.xlabel('Log-Lambda')

In [ ]:
# Plot partial slopes vs R squared (% deviance explained)
plt.plot(rsq_array, beta_coeffs)
plt.xlim([0.0, 0.25])
plt.title('Partial Slopes vs Log-Lambda')
plt.xlabel('R-squared')
plt.ylabel('Partial Slopes')

Notice that model coefficients are much more tightly constrianed than for L2 regularization. In fact, only two of the possible model coefficients have non-zero values at all. This is typical of L1 or lasso regression.

---------------
### Elastic net regression
---------------

The **elastic net** algorthm uses a weighted combination of L2 and L1 regularization. As you can probably see, the same function is used for Lasso and Ridge regression with only the `L1_wt` argument changing. This argument determines the how much weight goes to the L1-norm of the partial slopes. If `L1_wt = 0`, the regularization is pure L2 (Ridge) and if `L1_wt = 1.0` the regularization is pure L1 (Lasso).

The code in the cell below gives equal weight to each regression method. Excute this code and examine the results.

In [ ]:
# Elasticnet Regression with a sequence of lambdas
# Generate a sequence of lambdas
log_lambda_seq = np.linspace(-6, 0, 50)
lambda_seq = np.exp(log_lambda_seq)

coeffs_array = []
rsq_array = []
formula = 'childHeight ~ mother + father + mother_sqr + father_sqr + 1'

for lamb in lambda_seq:
    ridge_model = sm.ols(formula, data=male_df).fit_regularized(method='elastic_net', alpha=lamb, L1_wt=0.75)
    coeffs_array.append(list(ridge_model.params))
    predictions = ridge_model.fittedvalues
    residuals = [x - y for x, y in zip(np.squeeze(predictions), childHeight)]

    SSR = np.sum(np.square(residuals))
    SST = np.sum(np.square(childHeight - np.mean(childHeight)))

    rsq = 1 - (SSR / SST)
    rsq_array.append(rsq)

In [ ]:
# Pull out partial slopes (drop intercept version)
beta_coeffs = [x[1:] for x in coeffs_array]
plt.plot(log_lambda_seq, beta_coeffs)
plt.title('Partial Slopes vs Log-Lambda')
plt.ylabel('Partial Slope Values')
plt.xlabel('Log-Lambda')

In [ ]:
# Plot partial slopes vs R squared (% deviance explained)
plt.plot(rsq_array, beta_coeffs)
plt.xlim([0.0, 0.25])
plt.title('Partial Slopes vs Log-Lambda')
plt.xlabel('R-squared')
plt.ylabel('Partial Slopes')

Notice that the elastic net model combines some of the behaviors of both L2 and L1 regularization. 

## Categorical Variables and the Model Matrix

Up until now we have only been working with numeric data. How can we handle categorical variables in numeric models? 

We need to encode the categorical variables into one or more numeric variables. The common approach is to convert the categorical variable to a set of binary **dummy variables** or **indicator variables**. 

The code in the cell below computes the scaled model data frame. Execute this code and examine the summary of the data frame.

In [ ]:
# Full original Family dataset
family_data[0:3]

In [ ]:
# Create new data frame for new feature set
height_df = family_data.copy()

# Add in squares of mother and father heights
height_df['father_sqr'] = height_df['father'] **2
height_df['mother_sqr'] = height_df['mother'] **2

# Drop columns for family, kids
Obsolete = ["family", "kids"]
for x in Obsolete:
    height_df = height_df.drop(x, axis=1)
    
# Reset the index
male_df=male_df.reset_index(drop=True)

# preview the data
height_df.head()

In [ ]:
# Add scaled x to data frame
height_df['father'] = scale(height_df['father'])
height_df['mother'] = scale(height_df['mother'])
height_df['father_sqr'] = scale(height_df['father_sqr'])
height_df['mother_sqr'] = scale(height_df['mother_sqr'])

print(height_df.head())

In [ ]:
# Fit the OLS with the categorical field 'gender':
ols_model = sm.ols(formula = 'childHeight ~ father + mother + father_sqr + mother_sqr + gender + 1', data=height_df)

results = ols_model.fit()
n_points = height_df.shape[0]
y_output = height_df['childHeight'].reshape(n_points, 1)

# Get slope (m) and y-intercept (b)
print('Intercept, Slopes : \n{}'.format(results.params))

# Get the t-values (hypothesis test statistics) for linear regression coefficient hypothesis tests.
print('Intercept t-value, Slope t-values: \n{}'.format(results.tvalues))

# Get p-values for above t-value statistics
print('\nHypothesis test summary for each coefficient if they differ from zero:')
print(results.pvalues)

print('\nSSE, SST, SSR, and RMSE:')
mean_y = np.mean(y_output)
sst = np.sum((y_output - mean_y)**2)
sse = sst - results.ssr
print('SSE: {}'.format(sse))
print('SST: {}'.format(sst))
print('SSR: {}'.format(results.ssr))
print('RMSE: {}'.format(np.sqrt(results.mse_model)))

# Get most of the linear regression statistics we are interested in:
print(results.summary())

# Plot a histogram of the residuals
sns.distplot(results.resid, hist=True)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Residual Histogram')

------------------
## Total Regression
### Also known as Demming Regression or Orthagonal Distance Regression
------------------

Another case of regression we may be interested in is minimizing the total error.  By total error, we mean minimizing the error in the y-values _and_ the x-values.  Here, we will be interested in minimizing the distance between the point and the best fit line for all points.

A visual explanation of the distance we wish to minimize is in the following figure.  Regular regression minimizes the grey-vertical lines.  Total Regression will minimize the dashed red lines.

<img src="https://library.startlearninglabs.uw.edu/DATASCI410/img/total_vs_leastsquares.png" alt="Total Regression Errors" style="width: 400px;"/>

### When would you use Total Regression?

 - When you have uncertainty in both the y-values _and_ the x-values.
 - When both the x and y values are the same scales and units.
   - Note that it is recommended to scale both the y and x before total regression.
   - There are alternative methods here that will scale the distance metric to be proportional to both the x and y measurement scales. However, such methods are not covered in this class and left to the reader.  For more detail on how to accomplish this, read the documentation in the scipy docs here: https://docs.scipy.org/doc/scipy-0.18.1/reference/odr.html

In [ ]:
# Description of method:
#    https://docs.scipy.org/doc/scipy-0.18.1/reference/odr.html

from scipy.odr import Model, Data, ODR
from scipy.stats import linregress
import numpy as np

# Create test data
x = np.linspace(start=0, stop=20., num=25)
y = x + np.random.normal(size=25, scale=5)
my_data = Data(x, y)
my_data_df = pd.DataFrame({'x': x, 'y': y})

# Define a linear function.
def f(B, x):
    '''Linear function y = m*x + b'''
    # B is a vector of the parameters.
    # x is an array of the current x values.
    return B[0]*x + B[1]

# Instantiate the linear model
linear = Model(f)

# Create the model with initial parameter guesses
total_regression_model = ODR(my_data, linear, beta0=[1., 1.])

# Fit the model
total_reg_results = total_regression_model.run()

# Print results
total_reg_results.pprint()

total_slope = total_reg_results.beta[0]
total_intercept = total_reg_results.beta[1]
print('\nTotal Regression:')
print('\n  -Slope: {}'.format(total_slope))
print('\n  -Intercept: {}'.format(total_intercept))

total_y_fit = [total_slope * xi + total_intercept for xi in x]

Let's compare the results graphically to regular regression.

In [ ]:
ols_model = sm.ols(formula = 'y ~ x', data=my_data_df)

results = ols_model.fit()
n_points = my_data_df.shape[0]
y_output = my_data_df['y'].values.reshape(n_points, 1)

reg_slope = results.params[1]
reg_intercept = results.params[0]
print('\nRegular Regression:')
print('\n  -Slope: {}'.format(reg_slope))
print('\n  -Intercept: {}'.format(reg_intercept))

reg_y_fit = [reg_slope * xi + reg_intercept for xi in x]

print('\nSSE, SST, SSR, and RMSE:')
mean_y = np.mean(y_output)
sst = np.sum((y_output - mean_y)**2)
sse = sst - results.ssr
print('SSE: {}'.format(sse))
print('SST: {}'.format(sst))
print('SSR: {}'.format(results.ssr))
print('RMSE: {}'.format(np.sqrt(results.mse_model)))

# Get most of the linear regression statistics we are interested in:
# print(results.summary())

In [ ]:
from matplotlib import collections  as mc
import pylab as pl

# More terse equations:
m = total_slope
b = total_intercept

# Loop through and draw vertical + total regression errors
total_errs = []
reg_errs = []
for xi, yi, totaly, regy in zip(x, y, total_y_fit, reg_y_fit):
    # Total reg. error segment
    x_t = (yi + xi/m - b) / (m + 1/m)
    y_t = m * x_t + b
    temp_total_err = [(x_t, y_t), (xi, yi)]
    total_errs.append(temp_total_err)
    # Regular reg. error segment
    temp_reg_err = [(xi, regy), (xi, yi)]
    reg_errs.append(temp_reg_err)

reg_lc = mc.LineCollection(reg_errs, colors='blue')
total_lc = mc.LineCollection(total_errs, colors='red')
fig, ax = pl.subplots(figsize=(6, 10))
ax.add_collection(reg_lc)
ax.add_collection(total_lc)

plt.plot(x, y, 'o')
plt.plot(x, reg_y_fit, color='blue', label='Regular')
plt.plot(x, total_y_fit, color='red', label='Total')

plt.title('Regular vs Total Linear Regression')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()


Let's compare the SSR's:

In [ ]:
ssr_reg = results.ssr
ssr_total = np.sum([(yi - y_hat)**2 for yi, y_hat in zip(y, total_y_fit)])

print('Regular SSE: {}'.format(ssr_reg))
print('Total SSE: {}'.format(ssr_total))

This is as expected, because both SSR computations here based off the vertical distance.  Let's compute a new SSR for the total regression that is the sum-squared of the total-errors.

We have prior calculated all pairs of points, in the variable, `total_errs`.  Let's use that to compute a new SSR.

In [ ]:
squared_errs = []
for pair in total_errs:
    temp_squared_dist = (pair[0][1] - pair[1][1])**2 + (pair[0][0] - pair[1][0])**2
    squared_errs.append(temp_squared_dist)
    
new_total_ssr = np.sum(squared_errs)
print('New Total SSE: {}'.format(new_total_ssr))

> Note, however, it is harder to define the SST here, and as a result, any definition of an $R^{2}$ metric is arbitrary and not-comparable.

## Logistic Regression

Until now, we have been working strictly with linear regression models. Now we will look at a widely used variation on the linear model know as **logistic regression**.

Logistic regression is widely used as a classification model. Logistic regression is linear model, with a binary response, `{False, True}` or `{0, 1}`.  However, the response is computed as a log likelihood. In the simplest case, the response has a Binomial distribution. 

The response of the linear model is transformed to the log likelihood using a sigmodial function, also know as the **logistic function**:

$$f(x) = \frac{1}{1 + e^{-\kappa(x - x_0)}} \\
\kappa = steepness$$

Execute the code in the cell below to compute and plot an example of the logistic function.

In [ ]:
# Plot the logistic transformation function (f(x) above)
x_seq = np.linspace(-7, 7, 100)

def log_fun(x, center=0, scale=1):
    e = np.exp(-scale*(x-center))
    log_out = 1./(1. + e)
    return log_out

log_fun_vectorized = np.vectorize(log_fun)

log_y = log_fun_vectorized(x_seq)

plt.plot(x_seq, log_y)
plt.title('Standard Logistic Function')
plt.xlabel('x')
plt.ylabel('y')
plt.grid()

Let's make this a bit more concrete with a simple example. Say we have a linear model:

$$\hat{y} = \beta_0 + \beta_1\ x$$

Now, depending on the value of $\hat{y}$ we want to classify the output from a logistic regression model as either `0` or `1`. We can use the linear model in the logistic function as follows:

$$F(\hat{y}) = \frac{1}{1 + e^{-\kappa(\beta_0 + \beta_1\ x)}} $$

In this way we transform the continious output of the linear model defined on $-\infty \le \hat{y} \le \infty$ to a binary response, $0 \le F(\hat{y}) \le 1$

### Classification Example

Next, we will try to classify the gender of the children in the Galton families data set using logistic regression on the height data.

As a first step, we need to create a scaled model matrix of the features for the logistic regression. Run the code in the cell bellow to compute this matrix.

In [ ]:
# The logit function in Stats models does not take formulas,
#     instead we have to give it arrays/matrices.
#     This is very common with ML in python, so it is a good
#     time to get familiar with the format.

# Create intercept column
height_df['intercept'] = 1.0

X_cols = ['father', 'mother', 'childHeight', 'intercept']

X = height_df[X_cols]
Y = np.array([1 if x=='F' else 0 for x in height_df['gender']])

logit_model = sm.Logit(Y, X)

result = logit_model.fit()

In [ ]:
# Fix pending for issue in statsmodels:
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

print(result.summary())

We can generate a `alpha` level of confidence around the parameter estimates with the result method `results.conf_int(alpha=...)` as follows.

In [ ]:
# Confidence intervals
# alpha is the % confidence in parameters.
print(result.conf_int(alpha=0.90))

### Interpretation of parameters

If we are interested in how the output probabilities change with respect to the features, we can look at the odds ratio. The odds ratio is just the exponential of the parameters.

These tell us how a 1 unit increase or decrease in a variable affects the odds of being predicting a '1' (Female). For example, we can expect the odds of predicting a female gender to decrease by about 65.9% if we find the childHeight goes down by 1.

In [ ]:
# Odds ratio
print(np.exp(result.params))

Here is how we can make predictions with our model.

In [ ]:
# Standardized Father, Standardized Mother, ChildHeight, 1.0 (intercept)
test_input = np.array([3.7, 1.26, 70.1, 1.0])
test_output = result.predict(test_input)
print('Probability of Female: {0:.2f}%'.format(test_output[0]*100.))

Finding out the effect of `childHeight` on the probabilities graphically:

Here we will loop over a sequence of childHeights to see the effect of them on the probability of a Female gender.

In [ ]:
childHeight_seq = np.linspace(height_df['childHeight'].min(), height_df['childHeight'].max(), 100)

avg_father = height_df['father'].mean()
avg_mother = height_df['mother'].mean()

childHeight_df = pd.DataFrame(childHeight_seq, columns=['childHeight'])
childHeight_df['father'] = height_df['father'].mean() # hopefully close to zero
childHeight_df['mother'] = height_df['mother'].mean() # hopefully close to zero
childHeight_df['intercept'] = 1.0

# Rearrange order of columns appropriately
childHeight_df = childHeight_df[['father', 'mother', 'childHeight', 'intercept']]

# Get probability predictions
gender_prob_childHeight = result.predict(childHeight_df)

In [ ]:
plt.plot(childHeight_seq, gender_prob_childHeight)
plt.title('P(F) vs childHeight')
plt.xlabel('childHeight')
plt.ylabel('P(F)')
plt.grid()

We are also interested in evaluating our classifier's accuracy.  Here we will look at how many are:

 - True Positives (TP): We predicted Female and the actual gender was Female.
 - True Negatives (TN): We predicted Male and the actual gender was Male.
 - False Positives (FP): We predicted Female and the actual gender was Male.
 - False Negatives (FN): We predicted Male and the actual gender was Female.

The output of our predictions is a probability.  In order to make this a binary prediction (Female or male), we need to decide a cutoff.  Commonly, we choose 0.5 as a cutoff.  But know that this choice is arbitrary and we can set it to whatever probability we choose.  If we have a model where False Positives are costly, we might decide to increase the cutoff, and vice-versa if False Negatives are more costly.

In [ ]:
prediction_frame = X.copy()

prediction_frame['probability'] = result.predict(X)

prediction_frame['actual'] = Y

cutoff = 0.5
def prediction_fun(row):
    if row['probability'] > 0.5:
        return 1.0
    else:
        return 0.0
    

prediction_frame['prediction'] = prediction_frame.apply(prediction_fun, axis=1)

In [ ]:
prediction_frame.head()

Now let's calculate the accuracy, true positives, true negatives, false positives, and false negatives.

In [ ]:
TP = np.sum([a == 1.0 and p == 1.0 for a, p in zip(prediction_frame['actual'], prediction_frame['prediction'])])
TN = np.sum([a == 0.0 and p == 0.0 for a, p in zip(prediction_frame['actual'], prediction_frame['prediction'])])

FP = np.sum([a == 0.0 and p == 1.0 for a, p in zip(prediction_frame['actual'], prediction_frame['prediction'])])
FN = np.sum([a == 1.0 and p == 0.0 for a, p in zip(prediction_frame['actual'], prediction_frame['prediction'])])

num_obs = prediction_frame.shape[0]

print('Out of {} observations:'.format(num_obs))
print('TP: {}, TN: {}'.format(TP, TN))
print('\nFP: {}, FN: {}'.format(FP, FN))

accuracy = (TP + TN) / num_obs
print('\nAccuracy over two classes: {0:.2f}%'.format(accuracy*100.))

### Confusion Matrix
We can view these results in a table with a confusion matrix as follows:

In [ ]:
print('Actual = columns')
print('\nPredicted = rows')
print('\n   0.0   1.0')
print('----------------')
print(result.pred_table(threshold=cutoff))

In the confusion matrix, males are defined as Positive cases and females are Negative cases. Notice that most of the cases in this data are correctly classified with only a few false negatives and false positives.

The other metrics are defined as follows:

- Accuracy = (TP + TN) / (TP + FP + TN + FN)
- Precision = TP / (TP + FP)
   - Precision is the fraction of the relevant class predictions are actually correct.
- Recall = TP / (TP + FN)
   - Recall is the fraction of the relevant class were we able to predict.

These summary statistics show the classifier works fairly well.

### Summary
In this notebook, we looked at 3 methods as a solution to the problem of overfit models:

* Stepwise Regression to eliminate features one at a time
* Singular Value Decomposition to find meaningful features
* Lasso, Ridge, and Elastic-net Regularization to stabilize over-parameterized models.

Important terms from this notebook include:

* AIC - the model log-likelihood adjusted for the number of model parameters
* Deviance - a measure of the relative likelihood of the model (generalization of variance)
* Conformable - number of rows of first matrix equals the number of columns of the second matrix, number of columns of first matrix equals number of rows of the second matrix
* SVD - Singular Value Decomposition describes a matrix as linear combination of a series of vectors $U$, $V$, $D$
* singular values $s$ - the diagonal matrix of singular values used as a scaling term
* Rank deficient matrix - has one or more of the $m$ singular values  $d_i  \sim 0.0$  
* Lambda - a small bias term added to singular values to stabilize the inverse singular value matrix
* PCA - Principal Component Analysis to determine what new features can be made from linear combinations that are independent of each other by using functions and the SVD
* PCR - Principal Component Regression uses the PCA to perform a "regularized" regression.
* L2 Norm regularization - Ridge regularization limiting the Euclidean norm of values of the model coefficient vector
* L1 regularization - Lasso regularization using the Manhattan norm
* Elastic-net algorithm - a weighted combination of L2 and L1 regularization
* Total regression - minimizes the distance between the point and the best fit line for all points
* Logistic regression - a linear model, with a binary response (Binomial distribution) using the logistic function on the response to transform it into a log likelihood


And we reviewed Linear Algebra operations:
* adding vectors and matrices
* multiplying vectors and matrices
* transposing matrices with `np.transpose`
* dot product or scalar product or inner product with `np.dot`
* L2 norm of a vector
* identity matrix with `np.eye`
* inverse of a matrix with `np.linalg.inv`
* singular value decomposition with `np.linalg.svd`
* creating a diagonal matrix with `np.diag`




<div id="reminder" style="border-radius: 5px; background-color:#f5f5f5;" >
<h3>Reminder</h3>
<p>Use this notebook to answer the quiz questions related to the <b>Your Turn</b> sections.<p>
</div>